In [1]:
import keras
from keras.models import load_model
model_final =load_model('new_rcnn_vgg16_1.h5')

In [ ]:
import numpy as np
import cv2
import urllib.request
cv2.setUseOptimized(True);
ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
# for webcams, request stream only once.
path="http://192.168.29.98:8080/shot.jpg"
while(True):
    imgResp = urllib.request.urlopen(path)
    imgNp = np.array(bytearray(imgResp.read()), dtype=np.uint8)
    imgOriginal = cv2.imdecode(imgNp, -1)
    img = np.asarray(imgOriginal)
    img=cv2.resize(img,(224,224))
    img = cv2.resize(img, (224,224), interpolation = cv2.INTER_AREA)
    ss.setBaseImage(img)
    ss.switchToSelectiveSearchFast()
    ssresults = ss.process()
    imout = img.copy()
    for e,result in enumerate(ssresults):
        if e < 100:
            x,y,w,h = result
            timage = imout[y:y+h,x:x+w]
            resized = cv2.resize(timage, (224,224), interpolation = cv2.INTER_AREA)
            img = np.expand_dims(resized, axis=0)
            out= model_final.predict(img)
            if out[0][0] > 0.80:
                cv2.rectangle(imout, (x, y), (x+w, y+h), (0, 0, 255), 1, cv2.LINE_AA)
    cv2.imshow("frame",imout)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Done. release the capture
cap.release()
cv2.destroyAllWindows()